## Exploring LSTM Results

We will explore the results for one site in this notebook.
1. We need to get the batch data and the pick the approprate arrays from it
2. Next we need to get the average of all the forecasts - as the model is being run 5 times
3. Next we need to post-process the data to inverse scale it
4. Next plot it and see

In [13]:
import sys
sys.path.insert(0, '../')

import src.utils as util
%load_ext autoreload
%autoreload 2

import pandas as pd

In [107]:
from sklearn.preprocessing import StandardScaler
import numpy as np

In [20]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [24]:
from plotly.validators.scatter.marker import SymbolValidator

raw_symbols = SymbolValidator().values
namestems = []
namevariants = []
symbols = []
for i in range(0,len(raw_symbols),3):
    name = raw_symbols[i+2]
    symbols.append(raw_symbols[i])
    namestems.append(name.replace("-open", "").replace("-dot", ""))
    namevariants.append(name[len(namestems[-1]):])

In [146]:
symbols[18]

204

In [197]:
# read the files
site_id = 'grid'
fc = []
actual = []

for num_iter in range(1, 3):
    fc_iter = pd.read_pickle(f'../lstm_results/forecast_{site_id}_iteration_{num_iter}') 
    ac_iter = pd.read_pickle(f'../lstm_results/actual_{site_id}_iteration_{num_iter}') 
    
    fc.append(fc_iter)
    actual.append(ac_iter)

In [198]:
data = pd.read_pickle(f'../ts_data/{site_id}')

In [199]:
data.head()

,power
date_str,
2020-02-13 07:00:00,12.83
2020-02-13 08:00:00,8.70
2020-02-13 09:00:00,13068.87
2020-02-13 10:00:00,2788.11
2020-02-13 11:00:00,131494.68


In [200]:
look_back = 14 * 7  # 14 hours in to 7 days

# train, val, test split
train, val, test = util.split_hourly_data(data, look_back)

In [201]:
scaler = StandardScaler()
scaler.fit(train[['power']].values)

StandardScaler()

In [202]:
def extract_non_overlapping_samples(tf_data, scaler, horizon=14):
    extracted_samples = []
    array= tf_data[0]
    for i in range(1, len(tf_data)):
        array = np.concatenate((array, tf_data[i]), axis=0)
#         horizon_data = batch[::horizon]
#         for timepoint in horizon_data:
#             extracted_samples.extend(timepoint.tolist())
    
    horizon_data = array[::horizon]
    for horizon in horizon_data:
        extracted_samples.extend(horizon.tolist())
    return scaler.inverse_transform(extracted_samples)

In [203]:
# np.concatenate((actual[0][0], actual[0][1]), axis=0)

In [204]:
actual_samples = extract_non_overlapping_samples(actual[0], scaler)

In [205]:
fc_samples = extract_non_overlapping_samples(fc[0], scaler)

In [206]:
fc_samples2 = extract_non_overlapping_samples(fc[1], scaler)

In [207]:
len(actual_samples)

504

In [208]:
len(fc_samples)

504

In [209]:
# actual_samples

In [210]:
len(test[look_back:])

504

In [132]:
532-504

28

In [173]:
new_df = x= test[look_back:][['power']]

In [174]:
fc_samples[fc_samples<0] = 0

In [175]:
new_df['actual_nn'] = actual_samples

In [176]:
new_df['fc'] = fc_samples

In [177]:
new_df['fc1'] = fc_samples2

In [178]:
new_df["mean_fc"] = new_df.loc[: , "fc":"fc1"].mean(axis=1)

In [182]:
new_df.head()

,power,actual_nn,fc,fc1,mean_fc
date_str,,,,,
2021-01-09 07:00:00,0.00,-0.000024,0.000000,95.693839,47.846920
2021-01-09 08:00:00,9.91,9.909981,0.000000,119.616938,59.808469
2021-01-09 09:00:00,158.04,158.040004,152.196897,183.300614,167.748755
2021-01-09 10:00:00,960.20,960.199990,740.332610,647.510151,693.921380
2021-01-09 11:00:00,1652.86,1652.859996,1754.409223,1845.733825,1800.071524


In [164]:
new_df = new_df.resample("1H").mean()

In [165]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= new_df.index, y = new_df['power']))
fig.add_trace(go.Scatter(x= new_df.index, y = new_df['actual_nn']))
fig.add_trace(go.Scatter(x= new_df.index, y = new_df['fc'], name="forecast"))
fig.add_trace(go.Scatter(x= new_df.index, y = new_df['fc1'], name="forecast1"))